In [3]:
## Feed frame getters tests
import json
import feedparser
import requests
import transmissionrpc as tr
import pandas as pa
import numpy
import re
import celery

In [4]:
def reord(d, regex, has_episodes=False, has_seasons=False):
    d = d.reindex(columns=['title', 'published', 'size', 'magnet_uri'])
    d.sort(['published', 'title', 'size'], ascending=False, inplace=True)
    
    def map_title(regex, group_num, default_ret, title):
        ma = re.match(regex, title)
        if ma and re.compile(regex).groups >= group_num:
            ret = ma.group(group_num).strip()
        else:
            ret = default_ret
        return ret
    
    if has_episodes and has_seasons:
        season_group = 2
        episode_group = 3
    elif has_episodes:
        season_group = 42
        episode_group = 2
    else:
        season_group = 42
        episode_group = 42
    
    d['name'] = d['title'].map(lambda x: map_title(regex, 1, numpy.nan, x))
    d.dropna(inplace=True)
    d['episode'] = d['title'].map(lambda x: int(map_title(regex, episode_group, 0, x)))
    d['season'] = d['title'].map(lambda x: int(map_title(regex, season_group, 0, x)))
    d.drop_duplicates(['name', 'season', 'episode'], inplace=True)

    return d

def get_from_strike(url, regex, has_episodes=False, has_seasons=False):
    r = requests.get(url)
    j = json.loads(r.text)
    d = pa.DataFrame(j['torrents'])
    d.rename(columns={'torrent_title': 'title', 'upload_date': 'published'},
             inplace=True)
    d = reord(d, regex, has_episodes, has_seasons)
    return d

def get_from_kickass(url, regex, has_episodes=False, has_seasons=False):
    rss = feedparser.parse(url)
    d = pa.DataFrame(rss['entries'])
    d.rename(columns={'torrent_magneturi': 'magnet_uri',
                      'torrent_contentlength': 'size'}, inplace=True)
    d = reord(d, regex, has_episodes, has_seasons)
    return d
    
#d = get_from_strike('https://getstrike.net/api/v2/torrents/search/?phrase=HorribleSubs%201080p&category=Anime', '^.HorribleSubs.\s([a-zA-Z0-9\s\'!-]+)\s-\s(\d{1,2}).*1080p.*$', True, False)   
#d = get_from_kickass('http://kickass.to/usearch/HorribleSubs%201080p/?rss=1', '^.HorribleSubs.\s([a-zA-Z0-9\s\'!-]+)\s-\s(\d{1,2}).*1080p.*$', True, False)   




,title,published,size,magnet_uri,name,episode,season
1,[HorribleSubs] Tribe Cool Crew - 35 [1080p].mkv,"Sun, 14 Jun 2015 22:10:02 +0000",488175267,magnet:?xt=urn:btih:A02B10FF6E4FDFBD8FDFB08488...,Tribe Cool Crew,35,0
2,[HorribleSubs] Show By Rock!! - 11 [1080p].mkv,"Sun, 14 Jun 2015 20:13:30 +0000",767838997,magnet:?xt=urn:btih:7EFB59B08CDED683F1AF4AE9AC...,Show By Rock!!,11,0
3,[HorribleSubs] Hello!! Kiniro Mosaic - 11 [108...,"Sun, 14 Jun 2015 20:13:15 +0000",565629577,magnet:?xt=urn:btih:3DD08916DFEE185B5275116592...,Hello!! Kiniro Mosaic,11,0
4,[HorribleSubs] Grisaia no Rakuen - 09 [1080p].mkv,"Sun, 14 Jun 2015 17:10:02 +0000",568203406,magnet:?xt=urn:btih:48B3F360A5564FA70951ACC3B4...,Grisaia no Rakuen,9,0
5,[HorribleSubs] Yamada-kun and the Seven Witche...,"Sun, 14 Jun 2015 16:40:02 +0000",579044658,magnet:?xt=urn:btih:5EEF4C516641B22D3F1A00EADE...,Yamada-kun and the Seven Witches,10,0
6,[HorribleSubs] Ame-iro Cocoa - 11 [1080p].mkv,"Sun, 14 Jun 2015 15:40:02 +0000",65589875,magnet:?xt=urn:btih:4719AE9D4395B92F2359063586...,Ame-iro Cocoa,11,0
7,[HorribleSubs] Ghost in the Shell Arise - Alte...,"Sun, 14 Jun 2015 14:10:02 +0000",725723660,magnet:?xt=urn:btih:0A5930D006263A956E84E6A86A...,Ghost in the Shell Arise - Alternative Archite...,10,0
8,[HorribleSubs] Baby Steps 2 - 11 [1080p].mkv,"Sun, 14 Jun 2015 10:10:03 +0000",587836731,magnet:?xt=urn:btih:6A644CB3B4AA46427BA6AF39B8...,Baby Steps 2,11,0
9,[HorribleSubs] Arslan Senki - 11 [1080p].mkv,"Sun, 14 Jun 2015 09:10:02 +0000",797401688,magnet:?xt=urn:btih:C23BD4A6D4000852DFEF8B896B...,Arslan Senki,11,0
10,[HorribleSubs] Kuroko's Basketball 3 - 73 [108...,"Sun, 14 Jun 2015 04:25:02 +0000",1037096979,magnet:?xt=urn:btih:74A6F2B0EFD83554D265CDAD5A...,Kuroko's Basketball 3,73,0


ImportError: No module named celery